In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import analysis_pca

Ts = 1000
data_path = r'C:\Users\IBS\Desktop\python_code_N2'
save_path = r'C:\Users\IBS\Desktop\figures'

In [18]:
#For FN angle averaged PCA

import pre_processing

ppss = pre_processing.pre_processing(Ts, data_path, save_path)
[final_cat, data_cat, val_cat, dur_cat, rejectedXY_cat] = ppss.datasorting_orientation(1)
apca = analysis_pca.analysis_pca(Ts, data_cat, save_path)
data_cat_sm = apca.smoothing_data()
data = np.mean(data_cat_sm, axis=0)
PCA_total = apca.pca_analy(data)

Dimensions: 1, Variance explained: 75.06%
Dimensions: 2, Variance explained: 85.13%
Dimensions: 3, Variance explained: 90.34%
Dimensions: 4, Variance explained: 93.95%
Dimensions: 5, Variance explained: 96.15%
Dimensions to be reduced: 3


In [19]:
PCA_total

{'v': array([[ 3.56921112e-02,  6.86971427e-02,  7.15129844e-02, ...,
          2.23188134e-01,  1.92497169e-01,  1.82150004e-01],
        [ 5.64687435e-01,  1.39666927e-01, -1.30859521e-01, ...,
          4.25919533e-01, -9.03386482e-02, -4.19257496e-02],
        [ 1.97805521e-01,  6.86884333e-02, -5.31953603e-03, ...,
          1.97473456e-01,  9.09445588e-02,  4.01965998e-02],
        ...,
        [-9.50703756e-03, -2.14307088e-02,  1.03132697e-01, ...,
          3.24190352e-04,  9.15302328e-02, -3.02517427e-02],
        [ 2.67393665e-02, -8.06822541e-03,  1.77579385e-01, ...,
          2.28738514e-02,  2.27247006e-01, -1.02398735e-01],
        [ 1.37380462e-01, -3.76940583e-02,  6.78977487e-02, ...,
         -1.15318434e-01, -3.86509288e-02,  1.03784473e-01]],
       shape=(67, 67)),
 'p': array([[-5.59243649e-02,  2.02036556e-02,  4.04035745e-02, ...,
          5.54706407e-04, -4.10090908e-04,  2.25117972e-05],
        [-6.37006963e-02,  1.35556648e-02,  3.98692260e-02, ...,
     